## Introduction

This notebook demonstrates an OpenAPI service-based Retriever-Augmented Generation (RAG) application using a Qdrant vector database with Haystack.

Starting with a user prompt, we fetch the appropriate OpenAPI specification (OAS) from Qdrant, as well as the matching system message. These retrieved text documents are then injected into the Haystack pipeline, facilitating the dynamic invocation of specific OAS services. By merging any OAS service with Large Language Models (LLMs), this cutting-edge method opens a new phase in RAG systems, enabling the generation of deeply contextual responses from structured OAS data services.

Slides: https://tinyurl.com/3b8vvpf8

## 1. Setup
This notebook demos Haystack 2.x service based RAG, using openapi services.

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [1]:
!pip install --no-deps sentence-transformers
!pip install "grpcio-tools==1.42" jsonref openapi3 qdrant-haystack git+https://github.com/deepset-ai/haystack.git@fcd48d662c17893dea8e899023d72885bd721359

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.6 MB/s eta 0:00:00
  Cloning https://github.com/deepset-ai/haystack.git (to revision fcd48d662c17893dea8e899023d72885bd721359) to /tmp/pip-req-build-stq9tze3
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-stq9tze3
  Running command git rev-parse -q --verify 'sha^fcd48d662c17893dea8e899023d72885bd721359'
  Running command git fetch -q https://github.com/deepset-ai/haystack.git fcd48d662c17893dea8e899023d72885bd721359
  Running command git checkout -q fcd48d662c17893dea8e899023d72885bd721359
  Resolved https://github.com/deepset-ai/haystack.git to commit fcd48d662c17893dea8e899023d72885bd721359
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/21

In [2]:
import getpass
import os
import json
import jsonref
import requests
from typing import List, Dict, Any
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever

from haystack import Pipeline, Document
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack.components.converters import OpenAPIServiceToFunctions, OutputAdapter
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.others import Multiplexer
from haystack.dataclasses import ChatMessage, ByteStream

### 1.1 Let's collect the API keys for LLM provider

In [3]:
llm_providers = {"fireworks": {"api_base_url": "https://api.fireworks.ai/inference/v1",
                               "text_model": "accounts/fireworks/models/mixtral-8x7b-instruct",
                               "fc_model": "accounts/fireworks/models/firefunction-v1"},
                 "openai": {"api_base_url": "https://api.openai.com/v1",
                            "text_model": "gpt-4-turbo-preview",
                            "fc_model": "gpt-3.5-turbo"}
                 }

# change this value to another provider defined above - if needed
selected_provider = "openai"
llm_api_key = getpass.getpass(f"Enter {selected_provider} api key:")


Enter openai api key:··········


### 1.2 And OpenAPI services we'll use:

In [4]:
# https://github.com/settings/tokens
github_token = getpass.getpass("Enter your github token:")

Enter your github token:··········


## 2. Setup QdrantDocumentStore

QdrantDocumentStore is one of many vector DB integrations we offer via Haystack Core Integrations [project](https://github.com/deepset-ai/haystack-core-integrations/)

In [5]:
# Note: for the demo we are using Qdrant in local on-disk mode.
# For production use cases, a Docker container should be preferred.
# See https://qdrant.tech/documentation/ for more details

document_store = QdrantDocumentStore(
    path="./qdrant_1",
    index="Document",
    embedding_dim=768,
    recreate_index=True
  )

### 2.1 Add some helper functions we'll use in both indexing and retrieval pipelines

In [6]:
def prepare_fc(openai_functions_schema: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "tools": [{
            "type": "function",
            "function": openai_functions_schema
        }],
        "tool_choice": {
            "type": "function",
            "function": {"name": openai_functions_schema["name"]}
        }
    }

def create_docs(functions, specs, system_messages, service_credentials):
  docs = []
  for function, spec, system_message, service_credential in zip(functions, specs, system_messages, service_credentials):
    d = Document(content=json.dumps(function),
                 meta={"spec": json.dumps(spec),
                       "system_message": system_message,
                       "service_credential": service_credential}) #it's just a demo - relax
    docs.append(d)
  return docs

## 3. Index OpenAPI specs

Each indexed service Document contains the following descriptors:

* OpenAI function calling JSON
* OpenAPI specification
* System message for the service
* Service credentials

In [7]:
embedding_model = "sentence-transformers/all-mpnet-base-v2"

# Note: if you play with the notebook and add more OAS services you are likely to get better service inference
# results with jina-embeddings-v2-small-en embedding model. jina-embeddings-v2-small-en is an English, monolingual
# embedding model supporting 8192 sequence length which should be enough for our Document describing OpenAI
# functions, OAS spec, and sytem message stored.

# embedding_model = "jina-embeddings-v2-small-en"
# os.environ['HF_API_TOKEN'] = getpass.getpass("Enter Hugging Face api key:")

In [8]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("fetcher", LinkContentFetcher())
indexing_pipeline.add_component("mx", Multiplexer(List[ByteStream]))
indexing_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())
indexing_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model=embedding_model,
                                                                                 meta_fields_to_embed=["spec", "system_message", "service_credential"],
                                                                                 trust_remote_code=True))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.add_component("a1", OutputAdapter("{{functions | create_docs(specs, system_messages, service_credentials)}}",
                                                     List[Document],
                                                      {"create_docs": create_docs}))

indexing_pipeline.add_component("a2", OutputAdapter("{{sources | json_objects}}",
                                                     List[Dict[str, Any]],
                                                      {"json_objects": lambda sources: [json.loads(s.to_string()) for s in sources]}))

indexing_pipeline.connect("fetcher.streams", "mx")
indexing_pipeline.connect("mx", "spec_to_functions.sources")
indexing_pipeline.connect("mx", "a2.sources")
indexing_pipeline.connect("a2", "a1.specs")
indexing_pipeline.connect("spec_to_functions.functions", "a1")
indexing_pipeline.connect("a1.output", "embedder.documents")
indexing_pipeline.connect("embedder", "writer")

🚅 Components
  - fetcher: LinkContentFetcher
  - mx: Multiplexer
  - spec_to_functions: OpenAPIServiceToFunctions
  - embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
  - a1: OutputAdapter
  - a2: OutputAdapter
🛤️ Connections
  - fetcher.streams -> mx.value (List[ByteStream])
  - mx.value -> spec_to_functions.sources (List[ByteStream])
  - mx.value -> a2.sources (List[ByteStream])
  - spec_to_functions.functions -> a1.functions (List[Dict[str, Any]])
  - embedder.documents -> writer.documents (List[Document])
  - a1.output -> embedder.documents (List[Document])
  - a2.output -> a1.specs (List[Dict[str, Any]])

In [9]:
# indexing_pipeline.draw("/content/indexing_pipeline.png")

PipelineDrawingError: There was an issue with https://mermaid.ink/, see the stacktrace for details.

### 3.1 We'll use three OAS services: GitHub, SerperDev, TinyURL



*   GitHub - compare branches operation
*   SerperDev - a search engine bridge
*   TinyURL - URL shortener  



In [10]:
services = {"open_api_specs": ["https://bit.ly/github_compare"],
            "system_messages":["https://bit.ly/pr_auto_system"]}

In [12]:
result = indexing_pipeline.run(data={"fetcher": {"urls": services["open_api_specs"]},
                                     "a1":{"service_credentials": [github_token],
                                           "system_messages": [requests.get(url).text for url in services["system_messages"]]
                                           }
                                     })
print(f"\nWrote {result['writer']['documents_written']} documents")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100it [00:00, 8374.87it/s]           


Wrote 1 documents


## 4. Build the retrieval/LLM generation pipeline

In [13]:
from haystack.utils import Secret

pipe = Pipeline()
pipe.add_component("embedder", SentenceTransformersTextEmbedder(model=embedding_model, trust_remote_code=True))
pipe.add_component("retriever", QdrantEmbeddingRetriever(document_store=document_store))
pipe.add_component("mx", Multiplexer(List[Document]))
pipe.add_component("a1", OutputAdapter("{{documents[0].content | tojson | prepare_fc}}", Dict[str, Any], {"prepare_fc": prepare_fc, "tojson": lambda s: json.loads(s)}))
pipe.add_component("a2", OutputAdapter("{{documents[0].meta['spec'] | tojson}}", Dict[str, Any], {"tojson": lambda s: jsonref.replace_refs(json.loads(s))}))
pipe.add_component("a3", OutputAdapter("{{documents[0].meta['service_credential']}}", str))
pipe.add_component("a4", OutputAdapter("{{documents[0].meta['system_message'] | cm}}", List[ChatMessage], {"cm": lambda s: [ChatMessage.from_system(s)]}))
pipe.add_component("a5", OutputAdapter("{{system_message + service_response}}", List[ChatMessage], {"cm": lambda s: [ChatMessage.from_system(s)]}))
pipe.add_component("fn_llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key),
                                                 model=llm_providers[selected_provider]["fc_model"],
                                                 api_base_url=llm_providers[selected_provider]["api_base_url"]))
pipe.add_component("openapi_container", OpenAPIServiceConnector())
pipe.add_component("llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key),
                                              streaming_callback=print_streaming_chunk,
                                              api_base_url=llm_providers[selected_provider]["api_base_url"],
                                              generation_kwargs={"max_tokens": 1024},
                                              model=llm_providers[selected_provider]["text_model"],
                                              ))

pipe.connect("embedder", "retriever")
pipe.connect("retriever.documents", "mx")
pipe.connect("mx", "a1.documents")
pipe.connect("mx", "a2.documents")
pipe.connect("mx", "a3.documents")
pipe.connect("mx", "a4.documents")
pipe.connect("a1", "fn_llm.generation_kwargs")
pipe.connect("a2", "openapi_container.service_openapi_spec")
pipe.connect("a3", "openapi_container.service_credentials")
pipe.connect("a4", "a5.system_message")
pipe.connect("a5", "llm.messages")
pipe.connect("openapi_container.service_response", "a5.service_response")
pipe.connect("fn_llm.replies", "openapi_container.messages")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: QdrantEmbeddingRetriever
  - mx: Multiplexer
  - a1: OutputAdapter
  - a2: OutputAdapter
  - a3: OutputAdapter
  - a4: OutputAdapter
  - a5: OutputAdapter
  - fn_llm: OpenAIChatGenerator
  - openapi_container: OpenAPIServiceConnector
  - llm: OpenAIChatGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> mx.value (List[Document])
  - mx.value -> a1.documents (List[Document])
  - mx.value -> a2.documents (List[Document])
  - mx.value -> a3.documents (List[Document])
  - mx.value -> a4.documents (List[Document])
  - a1.output -> fn_llm.generation_kwargs (Dict[str, Any])
  - a2.output -> openapi_container.service_openapi_spec (Dict[str, Any])
  - a3.output -> openapi_container.service_credentials (str)
  - a4.output -> a5.system_message (List[ChatMessage])
  - a5.output -> llm.messages (List[ChatMessage])
  - fn_llm.replies -> openapi_container.messag

In [ ]:
# pipe.draw("/content/pipe.png")

## 5. Retrieval, service invocation and LLM generation

Here is where everything gets interesting! Based on the prompt below, we'll retrieve the top_k=1 document from QdrantDocumentStore. That'll be the relevant service Document. We'll then proceed to use service descriptors from the retrieved Document to initiate:

* Step 1: Function calling to resolve the parameters for service invocation.

<img src="https://raw.githubusercontent.com/vblagoje/various/main/OAS-1.1.png" width="600" alt="Step 1">

* Step 2: We'll invoke the OpenAPI-compliant service with those parameters.
<img src="https://raw.githubusercontent.com/vblagoje/various/main/OAS-2.png" width="600" alt="Step 1">

* Step 3: The response from the service will be joined with the system prompt for that service.
<img src="https://raw.githubusercontent.com/vblagoje/various/main/OAS-3.png" width="600" alt="Step 1">

* Step 4: The final LLM response will be generated.


This process involves straightforward intent recognition that utilizes vector similarity to accurately identify and retrieve the appropriate service Document. Theoretically, it's possible to have dozens of services indexed within the QdrantDocumentStore. Once we retrieve the right service Documents, we will rely on the Haystack pipeline to perform the magic and observe the response generated by the LLM.

In [14]:

user_prompt = "Compare Github branches main and test/benchmarks2.0, in project deepset-ai, repo haystack"
result = pipe.run(data={"fn_llm": {"messages":[ChatMessage.from_user(user_prompt)]}, "embedder": {"text": user_prompt}})


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

### Why:
The Pull Request (PR) aims to introduce significant additions and improvements to the benchmarking suite of the Haystack project by deepset-ai. It includes changes that enhance the project's capability to efficiently benchmark different components, specifically focusing on document recall evaluation and fixing dataset download issues, among other modifications. These changes are vital for continuously testing and validating the performance and reliability of Haystack components against various datasets and scenarios, ensuring the quality and robustness of the solutions provided to end-users.

### What:
- **Addition of GitHub Action Workflow for Benchmarks**: A new workflow file `.github/workflows/benchmarks-haystack-2.yml` has been added to automate the benchmarking process. This will trigger benchmark tests according to a defined schedule or manually through workflow dispatch, enabling systematic performance evaluation.
- **Metric Handling for DataDog**: New `metric_handler.p

## Links:
- https://github.com/deepset-ai/haystack/
- https://haystack.deepset.ai/community
- https://x.com/julian_risch
- https://x.com/vladblagoje

Solution: https://tinyurl.com/5r3ec39n